## 🔹 Mount Google Drive & Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 🔹 Objective

In [ ]:
# Implement CNN on BUILDINGS vs STREET image dataset.

## 🔹 Step 1: Import Libraries

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

## 🔹 Step 2: Dataset Paths & Parameters

In [ ]:
base_dir = '/content/drive/MyDrive/Assess-1-imageDataset'
img_size = 128
batch_size = 32
categories = ['BUILDINGS', 'STREET']

## 🔹 Step 3: Data Generators (Train, Validation)

In [ ]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(base_dir, target_size=(img_size, img_size),
                                        class_mode='binary', subset='training',
                                        batch_size=batch_size, shuffle=True, seed=42)

val_gen = datagen.flow_from_directory(base_dir, target_size=(img_size, img_size),
                                      class_mode='binary', subset='validation',
                                      batch_size=batch_size, shuffle=False, seed=42)

## 🔹 Step 4: Base Model

In [ ]:
def build_base_model():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

## 🔹 Step 5: Improved Model

In [ ]:
def build_improved_model():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(img_size, img_size, 3)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),

        Conv2D(64, (3,3), activation='relu', strides=2, padding='valid'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),

        Flatten(),
        Dropout(0.4),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
    return model

## 🔹 Step 6: Train Models

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model1 = build_base_model()
history1 = model1.fit(train_gen, epochs=15, validation_data=val_gen, callbacks=[es], verbose=1)

model2 = build_improved_model()
history2 = model2.fit(train_gen, epochs=15, validation_data=val_gen, callbacks=[es], verbose=1)

## 🔹 Step 7: Plot Accuracy and Loss

In [ ]:
def plot_metrics(hist, label):
    plt.plot(hist.history['val_accuracy'], label=f'{label} - Val Acc')
    plt.plot(hist.history['val_loss'], label=f'{label} - Val Loss')

plt.figure(figsize=(10,5))
plot_metrics(history1, 'Base')
plot_metrics(history2, 'Improved')
plt.title('Validation Accuracy & Loss')
plt.xlabel('Epochs')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

## 🔹 Step 8: Performance Comparison

In [ ]:
val_gen.reset()
y_true = val_gen.classes

y_pred1 = (model1.predict(val_gen) > 0.5).astype('int32').flatten()
y_pred2 = (model2.predict(val_gen) > 0.5).astype('int32').flatten()

print("🔸 Base Model Report:")
print(classification_report(y_true, y_pred1, target_names=categories))

print("🔸 Improved Model Report:")
print(classification_report(y_true, y_pred2, target_names=categories))

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
sns.heatmap(confusion_matrix(y_true, y_pred1), annot=True, fmt='d', cmap='Blues')
plt.title("Base Model Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")

plt.subplot(1,2,2)
sns.heatmap(confusion_matrix(y_true, y_pred2), annot=True, fmt='d', cmap='Greens')
plt.title("Improved Model Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()